# Pejorative

Creating `text_data.csv` and `annotations.csv` for Pejorative dataset.

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append('../../../')

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from datasets import load_dataset
from multitask_nlp.settings import DATASETS_DIR

tqdm.pandas()

dataset_path = DATASETS_DIR / 'pejorative_dataset'

In [2]:
df = pd.read_csv(dataset_path / 'tweet_datasets'/ 'English' / 'PEJOR1_annotated.csv', skiprows=2,
                names=['id', 'tweet', 'pejor_word', 'word_definition', 'annotator_1', 'annotator_2', 'annotator_3'])

In [3]:
df.head()

,id,tweet,pejor_word,word_definition,annotator_1,annotator_2,annotator_3
0,146,&#8220;@FreddieGibbs: Tea bag a bitch.&#8221; ...,bag,https://en.wiktionary.org/wiki/bag#English,0,0,NaN
1,288,@Sammy_Sicko Get me a bag of those pink n whit...,bag,https://en.wiktionary.org/wiki/bag#English,0,0,NaN
2,356,@itsjaylaaa eat shit nigga bitch turd bag dick...,bag,https://en.wiktionary.org/wiki/bag#English,1,1,NaN
3,548,I get a 50 pound bag of dawg for the mutts,bag,https://en.wiktionary.org/wiki/bag#English,0,0,NaN
4,618,Nuggas be to obsessed with the homies. Like th...,bag bitch,https://en.wiktionary.org/wiki/bag bitch#English,1,1,NaN


In [4]:
texts_df = df[['id', 'tweet', 'pejor_word']]
texts_df = texts_df.rename(columns={'id': 'text_id', 'tweet': 'text'})
texts_df = texts_df.drop_duplicates()

In [13]:
annotations_records = []

for i, row in df.iterrows():
    text_id = row.id
    for i in range(3):
        annotator_id = i+1
        annotation = row[f'annotator_{annotator_id}']
        
        if not pd.isna(annotation) and annotation in ['0', '1', '0.0', '1.0']:
            annotations_records.append({
                'text_id': text_id,
                'annotator_id': annotator_id,
                'is_pejorative': annotation
            }) 

annotations_df = pd.DataFrame.from_records(annotations_records)
annotations_df.is_pejorative = annotations_df.is_pejorative.astype('int64')

In [18]:
texts_df.to_csv(dataset_path / f'text_data.csv', index=False)
annotations_df.to_csv(dataset_path / f'annotations.csv', index=False)